# Etapa 1 (TASK 1)

A continuación se muestran las librerias usadas en esta primera parte del proyecto

In [6]:
import os
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re   # regular expressions
import string    
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.corpus import wordnet
from itertools import chain

C:\Users\fannb\Anaconda2022\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Carga de los datos

In [7]:
# loading and explore the data from the provided excel file
import os
wd = os.getcwd()
os.listdir(wd)

['.ipynb_checkpoints',
 'ColF and ColD TASK1_analysis.xlsx',
 'ColF and ColD_LancasterStem.xlsx',
 'ColF and ColD_PorterStem.xlsx',
 'ColF_ColD_TASK1.xlsx',
 'ColF_ColD_TASK1_clean.xlsx',
 'ColumnE_NACE_clean.xlsx',
 'ColumnF_LancasterStem.xlsx',
 'ColumnF_PorterStem.xlsx',
 'ColumnF_TASK1.xlsx',
 'ColumnF_TASK1_clean.xlsx',
 'Comercialización de estudios de mercado .xlsx',
 'Data_IAB.xlsx',
 'Data_IAB_clean.xlsx',
 'Data_NACE.xlsx',
 'Data_NACE_clean.xlsx',
 'First_match.xlsx',
 'IAB_LancasterStem.xlsx',
 'IAB_PorterStem.xlsx',
 'Proyecto de Comercializacion-Copy1.ipynb',
 'Proyecto de Comercializacion.ipynb',
 'Scripts for TASK2.ipynb',
 'Scripts for Task3.ipynb',
 'Second_TASK2_analysis.xlsx',
 'Subcategories_table.xlsx',
 'TASK1.xlsx',
 'TASK1_analysis.xlsx',
 'TASK1_clean.xlsx',
 'TASK1_LancasterStem.xlsx',
 'TASK1_PorterStem.xlsx',
 'TASK2_analysis.xlsx',
 'Task3.xlsx',
 'Workbook_Estudios_Mercado.xlsx',
 'Workbook_Estudios_Mercado_TASK1matched.xlsx',
 'Workbook_Estudios_Mercado_

Ahora voy a importar las librerias para manejo de excel en Python

In [7]:
'''
# how to create a simple spreadshet

from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet["A1"] = "hello"
sheet["B1"] = "World!"

workbook.save(filename="hello_world.xlsx")
'''

'\n# how to create a simple spreadshet\n\nfrom openpyxl import Workbook\n\nworkbook = Workbook()\nsheet = workbook.active\n\nsheet["A1"] = "hello"\nsheet["B1"] = "World!"\n\nworkbook.save(filename="hello_world.xlsx")\n'

In [8]:
# to read excel file

from openpyxl import load_workbook

wb = load_workbook(filename= 'Workbook_Estudios_Mercado.xlsx')


C:\Users\fannb\Anaconda2022\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
# to show the sheetnames
print(wb.sheetnames)

['TASK1', 'TASK2', 'TASK3', 'Models', 'IAB', 'NACE_REV2']


In [10]:
# create variables for TASK1 sheet, IAB and NACE sheets
sh = wb['TASK1']
sh_IAB = wb['IAB']
sh_NACE = wb['NACE_REV2']
sh, sh_IAB, sh_NACE

(<Worksheet "TASK1">, <Worksheet "IAB">, <Worksheet "NACE_REV2">)

In [11]:
type(sh)

openpyxl.worksheet.worksheet.Worksheet

I will create a dataset that stores the **IAB_codes** with the original text, the preprocessed text, and stemming texts.

In [12]:
import pandas as pd
import numpy as np

file_loc = 'Workbook_Estudios_Mercado.xlsx'
IAB_codes = pd.read_excel(file_loc, sheet_name= 'IAB', index_col=None, na_values = ['NA'], usecols=('A,B'))

IAB_codes.head()

,Name,Unique ID
0,Automotive,1
1,Auto Body Styles,2
2,Commercial Trucks,3
3,Sedan,4
4,Station Wagon,5


### Basic functions to access cells

In [13]:
# main cell object
sh['A1']

<Cell 'TASK1'.A1>

In [14]:
# to get the position of certain row and column
sh.cell(row=3, column=4)

<Cell 'TASK1'.D3>

In [15]:
# how to read a cell
sh['A1'].value

'isic_section'

In [16]:
# to get the value of a certain row and column
sh.cell(row=3, column=4).value

'Crop and animal production, hunting and related service activities'

In [17]:
# max number of rows in sheet
sh.max_row, sh_IAB.max_row

(145, 692)

## Data preprocessing

### stopwords

In [23]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

In [24]:
# mostrar las palabras del stopwords
print(stop_words)

{'do', 'be', 'into', 'had', 'any', 'o', 'all', 'we', 'between', 'once', 'not', "hadn't", 'does', 'these', 'd', 'been', 'yourselves', 'herself', "you've", 'from', 'why', 'll', 'other', 'me', 'only', "doesn't", 'or', 'ma', 'shouldn', 'a', 'they', 'itself', "couldn't", 'here', 'hadn', 'further', 'no', 'those', 'out', "she's", 'but', 's', 'couldn', "shan't", 'if', 'too', "should've", 'doing', 'as', 'than', 'how', 'can', 'just', "you'd", 'don', "aren't", 'after', 'of', 'our', 'very', 'there', 'mightn', 'himself', 'on', 'i', 'and', 'm', 'so', 'what', 'is', 'shan', 'nor', 'because', 'aren', 'am', 'over', 'its', 'under', 'whom', 'during', 'such', 'hers', 'his', 'in', 'was', 'hasn', 'will', 'more', 'at', 'them', 'when', 'by', 'same', "haven't", "that'll", 'weren', 'most', 'to', 'again', 'wouldn', 'that', 'until', 'are', "wouldn't", 've', 'didn', 'against', 'this', 'down', 'both', 'while', 'were', 'did', 'theirs', 'should', 'doesn', 'off', "wasn't", "won't", 'her', 'up', 'now', 'about', 'y', "mu

In [25]:
import re   # regular expressions
import string     
    

### working on TASK1 sheet

The following function was create just to extract the words of the column F for the initial analysis

In [26]:
def extract_words_single(row):
    # Extract strings from colums F(sk)
    cF = sh['F' + str(row)].value
    # convert all elements lower case
    cF = cF.lower()
    # remove punctuation and remove words containing numbers
    cF = re.sub('[%s]' % re.escape(string.punctuation), ' ', cF)
    # alfanumerico * times digits
    cF = re.sub('\w*\d\w*', ' ', cF)
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in cF.split() if x not in res]
    return res

In [27]:
import pandas as pd

In [28]:
data_cF = pd.DataFrame()
for i in range (2, sh.max_row + 1):
    data_cF = data_cF.append ({'Column F' : extract_words_single(i)}, ignore_index= True)
    data_cF.to_excel('ColumnF_TASK1.xlsx', sheet_name = 'test', index= False, freeze_panes=(1,1))

**data_cF** is a dataframe which contains only the column F. Planning to use it for a first matching.
Also an excel document **ColumnF_TASK1.xlsx** has been saved

In [29]:
data_cF.head(20)

,Column F
0,[farming]
1,[ranching]
2,[dairy]
3,[fishery]
4,"[mining, metals]"
5,"[oil, energy]"
6,"[defense, and, space]"
7,[pharmaceuticals]
8,"[food, production]"
9,"[aviation, and, aerospace]"


In [30]:
type(data_cF)

pandas.core.frame.DataFrame

I need to remove the stopwords from the previous dataframe.
The new data_clean does not contain the stop_words list.
Not expecting a difference because are isolated words.

In [31]:
# it works but I still want to keep tokens inside   
#data['clean_title'] = data['Full_text_TASK1'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))   
data_cF_clean = data_cF['Column F'].apply(lambda x: [item for item in x if item not in stop_words])

In [32]:
type(data_cF_clean)  #pandas.core.series.Series
data_cF_clean.head(20)


0                                   [farming]
1                                  [ranching]
2                                     [dairy]
3                                   [fishery]
4                            [mining, metals]
5                               [oil, energy]
6                            [defense, space]
7                           [pharmaceuticals]
8                          [food, production]
9                       [aviation, aerospace]
10                               [automotive]
11                                [chemicals]
12                                [machinery]
13                             [shipbuilding]
14                                 [textiles]
15                  [paper, forest, products]
16                    [railroad, manufacture]
17                                 [printing]
18    [electrical, electronic, manufacturing]
19                                 [plastics]
Name: Column F, dtype: object

to eliminate the ' ' in each element of the serie

In [33]:
data_cF_clean2 =[' '.join(tup) for tup in data_cF_clean]
data_cF_list = data_cF_clean2  # for using later in stemming
data_cF_clean2  # a list

['farming',
 'ranching',
 'dairy',
 'fishery',
 'mining metals',
 'oil energy',
 'defense space',
 'pharmaceuticals',
 'food production',
 'aviation aerospace',
 'automotive',
 'chemicals',
 'machinery',
 'shipbuilding',
 'textiles',
 'paper forest products',
 'railroad manufacture',
 'printing',
 'electrical electronic manufacturing',
 'plastics',
 'renewables environment',
 'glass ceramics concrete',
 'packaging containers',
 'industrial automation',
 'utilities',
 'construction',
 'building materials',
 'civil engineering',
 'cosmetics',
 'apparel fashion',
 'sporting goods',
 'tobacco',
 'supermarkets',
 'consumer electronics',
 'consumer goods',
 'furniture',
 'retail',
 'food beverages',
 'consumer services',
 'wholesale',
 'wine spirits',
 'luxury goods jewelry',
 'package freight delivery',
 'transportation trucking railroad',
 'warehousing',
 'airlines aviation',
 'maritime',
 'logistics supply chain',
 'import export',
 'hospitality',
 'restaurants',
 'computer hardware',
 'c

In [34]:
data_cF_clean2 = pd.DataFrame(data_cF_clean2, columns = ['Column F'])
data_cF_clean2.head()

,Column F
0,farming
1,ranching
2,dairy
3,fishery
4,mining metals


In [35]:
data_cF_clean2.to_excel('ColumnF_TASK1_clean.xlsx', sheet_name = 'Column F', index= False, freeze_panes=(1,1))

Another function to extract the column F y D from TASK1 tab (this is for evaluating other case, and to try to get a better match).

In [36]:
def extract_words_partial(row):
    # Extract strings from colums D(NACE),F(sk)
    cD = sh['D' + str(row)].value
    cF = sh['F' + str(row)].value
    # I will keep the column F at the beginning of the list
    c_full_text = ' '.join([cF, cD])
    # convert all elements lower case
    c_full_text = c_full_text.lower()
    # remove punctuation and remove words containing numbers
    c_full_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', c_full_text)
    # alfanumerico * times digits
    c_full_text = re.sub('\w*\d\w*', ' ', c_full_text)
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in c_full_text.split() if x not in res]
    return res

In [37]:
data_cF_cD = pd.DataFrame()
for i in range (2, sh.max_row + 1):
    data_cF_cD = data_cF_cD.append ({'ColF ColD' : extract_words_partial(i)}, ignore_index= True)
    data_cF_cD.to_excel('ColF_ColD_TASK1.xlsx', sheet_name = 'ColF ColD', index= False, freeze_panes=(1,1))


In [38]:
data_cF_cD.head(5)

,ColF ColD
0,"[farming, crop, and, animal, production, hunti..."
1,"[ranching, crop, and, animal, production, hunt..."
2,"[dairy, crop, and, animal, production, hunting..."
3,"[fishery, fishing, and, aquaculture]"
4,"[mining, metals, of, coal, and, lignite]"


In [39]:
type(data_cF_cD)

pandas.core.frame.DataFrame

In [40]:
# it works but I still want to keep tokens inside   
#data['clean_title'] = data['Full_text_TASK1'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))   
data_cF_cD_clean = data_cF_cD['ColF ColD'].apply(lambda x: [item for item in x if item not in stop_words])

In [41]:
type(data_cF_cD_clean)  #pandas.core.series.Series
data_cF_cD_clean.head(5)

0    [farming, crop, animal, production, hunting, r...
1    [ranching, crop, animal, production, hunting, ...
2    [dairy, crop, animal, production, hunting, rel...
3                      [fishery, fishing, aquaculture]
4                      [mining, metals, coal, lignite]
Name: ColF ColD, dtype: object

In [42]:
data_cF_cD_clean2 =[' '.join(tup) for tup in data_cF_cD_clean]
data_cF_cD_list = data_cF_cD_clean2  # for using later in stemming
data_cF_cD_clean2  # a list

['farming crop animal production hunting related service activities',
 'ranching crop animal production hunting related service activities',
 'dairy crop animal production hunting related service activities',
 'fishery fishing aquaculture',
 'mining metals coal lignite',
 'oil energy extraction crude petroleum natural gas',
 'defense space manufacture fabricated metal products except machinery equip',
 'pharmaceuticals manufacture medicinal chemical botanical products',
 'food production manufacture products',
 'aviation aerospace manufacture transport equipment',
 'automotive manufacture motor vehicles trailers semi',
 'chemicals manufacture chemical products',
 'machinery manufacture equipment n e c',
 'shipbuilding manufacture transport equipment',
 'textiles manufacture',
 'paper forest products manufacture',
 'railroad manufacture transport equipment',
 'printing reproduction recorded media',
 'electrical electronic manufacturing manufacture computer optical products',
 'plastics 

In [43]:
data_cF_cD_clean2 = pd.DataFrame(data_cF_cD_clean2, columns = ['ColF ColD'])
data_cF_cD_clean2.head()

,ColF ColD
0,farming crop animal production hunting related...
1,ranching crop animal production hunting relate...
2,dairy crop animal production hunting related s...
3,fishery fishing aquaculture
4,mining metals coal lignite


In [44]:
type(data_cF_cD)

pandas.core.frame.DataFrame

In [45]:
data_cF_cD_clean2.to_excel('ColF_ColD_TASK1_clean.xlsx', sheet_name = 'ColF and ColD', index= False, freeze_panes=(1,1))

The following function will add some columns into list. All the words will be converted to lower case, the punctuation and numbers and duplicates will be removed. Finally the list will be tokenized/split. 

In [56]:
def extract_words(row):
    # Extract strings from colums B(isic),D(NACE),F(sk),H(sk,gral group)
    cB = sh['B' + str(row)].value
    cD = sh['D' + str(row)].value
    cF = sh['F' + str(row)].value
    cH = sh['H' + str(row)].value
    # I will keep the column F at the beginning of the list
    c_full_text = ' '.join([cF, cH, cB, cD])
    # convert all elements lower case
    c_full_text = c_full_text.lower()
    # remove punctuation and remove words containing numbers
    c_full_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', c_full_text)
    # alfanumerico * times digits 
    c_full_text = re.sub('\w*\d\w*', ' ', c_full_text)
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in c_full_text.split() if x not in res]
    return res
    

data is a dataframe which contains each line of words of TASK. The range goes from line 2 (line 1 is a header) to the max amount of rows +1.
Inside the for the extract_words function is called.
An excel file with the TASK1.xlsx is created with only one sheet name.

In [57]:
data = pd.DataFrame()
for i in range (2, sh.max_row + 1):
    data = data.append ({'Full_text_TASK1' : extract_words(i)}, ignore_index= True)
    data.to_excel('TASK1.xlsx', sheet_name = 'test', index= False, freeze_panes=(1,1))

In [58]:
# to display with the max colwidth the following tables
pd.set_option('display.max_colwidth', None)

In [59]:
data # pandas DataFrame

,Full_text_TASK1
0,"[farming, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
1,"[ranching, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
2,"[dairy, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
3,"[fishery, agriculture, a, forestry, and, fishing, aquaculture]"
4,"[mining, metals, energy, and, b, quarrying, of, coal, lignite]"
...,...
139,"[individual, and, family, services, nonprofit, s, other, service, activities, personal]"
140,"[religious, institutions, nonprofit, s, other, service, activities, of, professional, membership, organizations]"
141,"[civic, social, organization, nonprofit, s, other, service, activities, of, professional, membership, organizations]"
142,"[non, profit, organization, management, nonprofit, s, other, service, activities, of, professional, membership, organizations]"


In [60]:
data.head(5)

,Full_text_TASK1
0,"[farming, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
1,"[ranching, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
2,"[dairy, agriculture, a, forestry, and, fishing, crop, animal, production, hunting, related, service, activities]"
3,"[fishery, agriculture, a, forestry, and, fishing, aquaculture]"
4,"[mining, metals, energy, and, b, quarrying, of, coal, lignite]"


In [61]:
type(data)

pandas.core.frame.DataFrame

In [62]:
# visualize the key data 
next(iter(data.keys()))

'Full_text_TASK1'

I need to remove the stopwords from the previous dataframe.
The new data_clean does not contain the stop_words list

In [63]:
# it works but I still want to keep tokens inside   
#data['clean_title'] = data['Full_text_TASK1'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))   
data_clean = data['Full_text_TASK1'].apply(lambda x: [item for item in x if item not in stop_words])

In [64]:
data_clean.head(5) # series


0     [farming, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]
1    [ranching, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]
2       [dairy, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]
3                                                         [fishery, agriculture, forestry, fishing, aquaculture]
4                                                          [mining, metals, energy, b, quarrying, coal, lignite]
Name: Full_text_TASK1, dtype: object

In [65]:
type(data_clean)

pandas.core.series.Series

I will concatenate the word to create a single string for each row


Not sure why this my **data is a data frame** and **my data_clean is a serie** ? Is it important to fixed?**

However, I figured out later that I can create a dataframe of the same data_clean. Example below **data_clean2**.

In [66]:
data_clean2 =[' '.join(tup) for tup in data_clean]
data_clean2

['farming agriculture forestry fishing crop animal production hunting related service activities',
 'ranching agriculture forestry fishing crop animal production hunting related service activities',
 'dairy agriculture forestry fishing crop animal production hunting related service activities',
 'fishery agriculture forestry fishing aquaculture',
 'mining metals energy b quarrying coal lignite',
 'oil energy mining b quarrying extraction crude petroleum natural gas',
 'defense space manufacturing c manufacture fabricated metal products except machinery equip',
 'pharmaceuticals healthcare c manufacturing manufacture medicinal chemical botanical products',
 'food production manufacturing c manufacture products',
 'aviation aerospace manufacturing c manufacture transport equipment',
 'automotive manufacturing c manufacture motor vehicles trailers semi',
 'chemicals manufacturing c manufacture chemical products',
 'machinery manufacturing c manufacture equipment n e',
 'shipbuilding manuf

In [67]:
type(data_clean2) # it is a list

list

now I will create a dataframe again 

In [68]:
data_clean2 = pd.DataFrame(data_clean2, columns = ['Full_text_TASK1'])
data_clean2.head(5)

,Full_text_TASK1
0,farming agriculture forestry fishing crop animal production hunting related service activities
1,ranching agriculture forestry fishing crop animal production hunting related service activities
2,dairy agriculture forestry fishing crop animal production hunting related service activities
3,fishery agriculture forestry fishing aquaculture
4,mining metals energy b quarrying coal lignite


Save data_clean as TASK1_clean.xlsx

In [69]:
data_clean2.to_excel('TASK1_clean.xlsx', sheet_name = 'Full_text_TASK1', index= False, freeze_panes=(1,1))

### working on IAB sheet

The following function will add some columns into list. All the words will be converted to lower case, the punctuation and numbers and duplicates will be removed. Finally the list will be tokenized/split. 

Some cells of IAB are empty, and it produced an error for certain row, therefore if elif else was used (I could not find an elegant form to do it)

In [70]:
def extract_words_IAB(row):
    # Extract strings from colums IAB
    cA = sh_IAB['A' + str(row)].value
    cD = sh_IAB['D' + str(row)].value
    cE = sh_IAB['E' + str(row)].value
    cF = sh_IAB['F' + str(row)].value
    #c = c1 + c3 + c5 + c7
    if cF == None and cA != cF and cD != cF and cE != cF :
        c_full_text = ' '.join([cA, cD, cE])
    elif cE == None and cA != cE and cD != cE:
        c_full_text = ' '.join([cA, cD])
    elif cD == None:
        c_full_text = ' '.join([cA])
    else:
        c_full_text = ' '.join([cA, cD, cE, cF])
    # convert all elements lower case
    c_full_text = c_full_text.lower()
    # remove punctuation and remove words containing numbers
    c_full_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', c_full_text)
    # alfanumerico * times digits
    c_full_text = re.sub('\w*\d\w*', ' ', c_full_text)
    # word separation
    #c_full_text.split()
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in c_full_text.split() if x not in res]
    return res

data_IAB is a dataframe which contains each line of words of IAB. The range goes from line 2 (line 1 is a header) to the max amount of rows +1.
Inside the for the extract_words function is called.
An excel file with the Data_IAB.xlsx is created with only one sheet name.

In [71]:
# it takes a while this run

data_IAB = pd.DataFrame()
for i in range (2, sh_IAB.max_row + 1):
    data_IAB = data_IAB.append ({'Text_IAB' : extract_words_IAB(i)}, ignore_index= True)
    data_IAB.to_excel('Data_IAB.xlsx', sheet_name = 'IAB', index= False, freeze_panes=(1,1))

In [72]:
data_IAB.head() #data frame

,Text_IAB
0,[automotive]
1,"[auto, body, styles, automotive]"
2,"[commercial, trucks, automotive, auto, body, styles]"
3,"[sedan, automotive, auto, body, styles]"
4,"[station, wagon, automotive, auto, body, styles]"


In [73]:
type(data_IAB)

pandas.core.frame.DataFrame

I need to remove the stopwords from the previous data_IAB. The new data_IAB_clean does not contain the stop_words list

In [74]:
data_IAB_clean = data_IAB['Text_IAB'].apply(lambda x: [item for item in x if item not in stop_words])
data_IAB_clean, type(data_IAB_clean)

(0                                                [automotive]
 1                            [auto, body, styles, automotive]
 2        [commercial, trucks, automotive, auto, body, styles]
 3                     [sedan, automotive, auto, body, styles]
 4            [station, wagon, automotive, auto, body, styles]
                                 ...                          
 686               [adult, video, games, gaming, game, genres]
 687    [casino, gambling, video, games, gaming, game, genres]
 688              [family, video, games, gaming, game, genres]
 689              [horror, video, games, gaming, game, genres]
 690              [racing, video, games, gaming, game, genres]
 Name: Text_IAB, Length: 691, dtype: object,
 pandas.core.series.Series)

Now I want to concatenate the words in each row.

In [75]:
data_IAB_clean2 = [' '.join(tup) for tup in data_IAB_clean]
data_IAB_clean2

['automotive',
 'auto body styles automotive',
 'commercial trucks automotive auto body styles',
 'sedan automotive auto body styles',
 'station wagon automotive auto body styles',
 'suv automotive auto body styles',
 'van automotive auto body styles',
 'convertible automotive auto body styles',
 'coupe automotive auto body styles',
 'crossover automotive auto body styles',
 'hatchback automotive auto body styles',
 'microcar automotive auto body styles',
 'minivan automotive auto body styles',
 'road vehicles automotive auto body styles',
 'pickup trucks automotive auto body styles',
 'auto type automotive',
 'budget cars automotive auto type',
 'certified pre owned cars automotive auto type',
 'classic cars automotive auto type',
 'concept cars automotive auto type',
 'driverless cars automotive auto type',
 'green vehicles automotive auto type',
 'luxury cars automotive auto type',
 'performance cars automotive auto type',
 'car culture automotive',
 'dash cam videos automotive',
 '

In [76]:
type(data_IAB_clean2) #list

list

convert the list to dataframe

In [77]:
data_IAB_clean2 = pd.DataFrame(data_IAB_clean2, columns = ['Text_IAB'])

In [78]:
data_IAB_clean2.head() # DataFrame

,Text_IAB
0,automotive
1,auto body styles automotive
2,commercial trucks automotive auto body styles
3,sedan automotive auto body styles
4,station wagon automotive auto body styles


In [79]:
type(data_IAB_clean2)

pandas.core.frame.DataFrame

In [80]:
data_IAB_clean2.to_excel('Data_IAB_clean.xlsx', sheet_name = 'IAB', index= False, freeze_panes=(1,1))

I will update my **IAB_codes**

In [81]:
IAB_codes

,Name,Unique ID
0,Automotive,1
1,Auto Body Styles,2
2,Commercial Trucks,3
3,Sedan,4
4,Station Wagon,5
...,...,...
686,Adult Video Games,MQ2XML
687,Casino and Gambling Video Games,ZJG29S
688,Family Video Games,VWGKS7
689,Horror Video Games,II436J


In [82]:
IAB_codes['IAB_clean'] = data_IAB_clean2['Text_IAB']
IAB_codes.head()

,Name,Unique ID,IAB_clean
0,Automotive,1,automotive
1,Auto Body Styles,2,auto body styles automotive
2,Commercial Trucks,3,commercial trucks automotive auto body styles
3,Sedan,4,sedan automotive auto body styles
4,Station Wagon,5,station wagon automotive auto body styles


### working on NACE sheet

The following function will add some columns into list. All the words will be converted to lower case, the punctuation and numbers and duplicates will be removed. Finally the list will be tokenized/split. 

the following function is for extracting just the column E for NACE. The last line of the NACE tab has several descriptions without a code, these descriptions are the section name. Be aware that some of the matching can be linked to these section names.
AGRICULTURE, FORESTRY AND FISHING
MINING AND QUARRYING
MANUFACTURING
ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY
WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES
CONSTRUCTION
WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES
TRANSPORTATION AND STORAGE
ACCOMMODATION AND FOOD SERVICE ACTIVITIES
INFORMATION AND COMMUNICATION
FINANCIAL AND INSURANCE ACTIVITIES
REAL ESTATE ACTIVITIES
PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES
ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES
PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY
EDUCATION
HUMAN HEALTH AND SOCIAL WORK ACTIVITIES
ARTS, ENTERTAINMENT AND RECREATION
OTHER SERVICE ACTIVITIES
ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS- AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE
ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES


In [ ]:
sh_NACE

In [ ]:
def extract_words_cE_NACE(row):
    # Extract strings from column E NACE
    cE = sh_NACE['E' + str(row)].value
    # convert all elements lower case
    cE = cE.lower()
    # remove punctuation and remove words containing numbers
    cE = re.sub('[%s]' % re.escape(string.punctuation), ' ', cE)
    # alfanumerico * times digits
    cE = re.sub('\w*\d\w*', ' ', cE)
    # word separation
    #cE.split()
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in cE.split() if x not in res]
    return res

In [ ]:
data_cE_NACE = pd.DataFrame()

In [ ]:
for i in range (2, sh_NACE.max_row + 1):
    data_cE_NACE = data_cE_NACE.append ({'Column E' : extract_words_cE_NACE(i)}, ignore_index= True)   

In [ ]:
data_cE_NACE.head(10), type(data_cE_NACE)

In [ ]:
data_cE_NACE_clean = data_cE_NACE['Column E'].apply(lambda x: [item for item in x if item not in stop_words ])


In [ ]:
type(data_cE_NACE_clean)

In [ ]:
data_cE_NACE_clean.head(10)

In [ ]:
# to eliminate the ' ' in each element of the serie
data_cE_NACE_clean2 = [' '.join(tup) for tup in data_cE_NACE_clean]

In [ ]:
data_cE_NACE_clean2 # a list

In [ ]:
data_cE_NACE_clean2 = pd.DataFrame(data_cE_NACE_clean2, columns = ['Column E'])
data_cE_NACE_clean2.to_excel('ColumnE_NACE_clean.xlsx', sheet_name='NACE', index= False, freeze_panes=(1,1))

Now, a dataframe with several columns will be created. Some cells of NACE are empty, and it produced an error for certain rows, therefore if elif else was used (I could not find an elegant form to do it)

In [ ]:
def extract_words_NACE(row):
    # Extract strings from columns NACE
    cE = sh_NACE['E' + str(row)].value
    cF = sh_NACE['F' + str(row)].value
    cG = sh_NACE['G' + str(row)].value
    cH = sh_NACE['H' + str(row)].value
    if cH == None and cE != cH and cF != cH and cG != cH :
        c_full_text = ' '.join([cE, cF, cG])
    elif cG == None and cE != cG and cF != cG:
        c_full_text = ' '.join([cE, cF])
    elif cF == None:
        c_full_text = ' '.join([cE])
    else:
        c_full_text = ' '.join([cE, cF, cG, cH])
    # convert all elements lower case
    c_full_text = c_full_text.lower()
    # remove punctuation and remove words containing numbers
    c_full_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', c_full_text)
    # alfanumerico * times digits
    c_full_text = re.sub('\w*\d\w*', ' ', c_full_text)
    # word separation
    #c_full_text.split()
    # remove duplicates and word separation
    res = []
    [res.append(x) for x in c_full_text.split() if x not in res]
    return res

data_NACE is a dataframe which contains each line of words of NACE. The range goes from line 2 (line 1 is a header) to the max amount of rows +1.
Inside the for the extract_words function is called.
An excel file with the Data_NACE.xlsx is created with only one sheet name.

In [ ]:
# it takes a while this run

data_NACE = pd.DataFrame()
for i in range (2, sh_NACE.max_row + 1):
    data_NACE = data_NACE.append ({'Text_NACE' : extract_words_NACE(i)}, ignore_index= True)
    data_NACE.to_excel('Data_NACE.xlsx', sheet_name = 'NACE', index= False, freeze_panes=(1,1))

In [ ]:
data_NACE.head(5)  #dataFrame

In [ ]:
type(data_NACE)

I need to remove the stopwords from the previous data_IAB. The new data_IAB_clean does not contain the stop_words list

In [ ]:
data_NACE_clean = data_NACE['Text_NACE'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
data_NACE_clean.head(), type(data_NACE_clean)  # serie

Now I will concatenate the words of each row.

In [ ]:
data_NACE_clean2 = [' '.join(i) for i in data_NACE_clean]
data_NACE_clean2  # a list

Convert the list to dataframe

In [ ]:
data_NACE_clean2 = pd.DataFrame(data_NACE_clean2, columns = ['Text_NACE'])

In [ ]:
data_NACE_clean2.head(5)

In [ ]:
type(data_NACE_clean2)

In [ ]:
data_NACE_clean2.to_excel('Data_NACE_clean.xlsx', sheet_name = 'NACE', index= False, freeze_panes=(1,1))

it has been created in this section the following variables and files:

- **sh, sh_IAB** sheet variables
- **data_cF** is a dataframe of column F in task1 and its excel is **ColumnF_TASK1.xlsx**
- **data_cF_clean** is a panda series, but several words separated with ' ' exist.
- **data_cF_clean2** is a dataframe with concatenated words in each row of Column F and its excel is **ColumnF_TASK1_clean.xlsx**
- **data_cF_list** = data_cF_clean2  # for using later in stemming

- **data_cF_cD_clean** is a panda series, with ' ' word separation.
- **data_cF_cD_clean2** is a dataframe with concatenated words in each cell of merge Col F and Col D and its **ColF_ColD_TASK1.xlsx**

- **data** is a dataframe of several column joined of TASK1 and its excel is **TASK1.xlsx**
- **data_clean** is a panda series, but several words separated with ' ' exist.
- **data_clean2** is a dataframe with concatenated words in each roach of Column F and its excel is **TASK1_clean.xlsx**
- **data_IAB** is a dataframe of IAB and its excel is **Data_IAB.xlsx**
- **data_IAB_clean** is a panda series and misses the concatenation of words in each row 
- **data_IAB_clean2** is a dataframe with concatenated words for each row of IAB and its excel is **Data_IAB_clean.xlsx**

- **data_cE_NACE** is a dataframe of **column E NACE**
- **data_cE_NACE_clean** is a panda series and misses the concatenation of words in each row
- **data_cE_NACE_clean2** is a dataframe with concatenated words of colE for each row of NACE and its excel is **ColumnE_NACE_clean.xlsx**
- **data_NACE** is a dataframe of NACE with **several merged cells** and its excel is **Data_NACE.xlsx**
- **data_NACE_clean** is a panda series and misses the concatenation of words in each row 
- **data_NACE_clean2** is a dataframe with concatenated words for each row of NACE and its excel is **Data_NACE_clean.xlsx**

Those underscore clean has been cleaned removing punctuation, digits and stop_words.


## Stemming

Some info
**PorterStemmer** uses Suffix Stripping to produce stems. 
PorterStemmer algorithm does not follow linguistics rather a set of 05 rules for different cases that are applied in phases (step by step) to generate stems.
This is the reason why PorterStemmer does not often generate stems that are actual English words, its simplicity and speed.

Porter’s Stemmer algorithm 
It is one of the most popular stemming methods proposed in 1980. It is based on the idea that the suffixes in the English language are made up of a combination of smaller and simpler suffixes. This stemmer is known for its speed and simplicity. The main applications of Porter Stemmer include data mining and Information retrieval. However, its applications are only limited to English words. Also, the group of stems is mapped on to the same stem and the output stem is not necessarily a meaningful word. The algorithms are fairly lengthy in nature and are known to be the oldest stemmer.
Example: EED -> EE means “if the word has at least one vowel and consonant plus EED ending, change the ending to EE” as ‘agreed’ becomes ‘agree’. 
 
Advantage: It produces the best output as compared to other stemmers and it has less error rate.
Limitation:  Morphological variants produced are not always real words. 
from https://www.geeksforgeeks.org/introduction-to-stemming/

**Lancaster Stemmer** is the most aggressive stemming algorithm. It has an edge over other stemming techniques because it offers us the functionality to add our own custom rules in this algorithm when we implement this using the NLTK package. This sometimes results in abrupt results.

The Lancaster stemmers are more aggressive and dynamic compared to the other two stemmers. The stemmer is really faster, but the algorithm is really confusing when dealing with small words. But they are not as efficient as Snowball Stemmers. The Lancaster stemmers save the rules externally and basically uses an iterative algorithm. 
from https://www.geeksforgeeks.org/introduction-to-stemming

In [83]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

ps = PorterStemmer() 

lan = LancasterStemmer()

In [84]:
'''
# example found
word_list = ["friend", "friendship", "friends", "friendships","stabil","destabilize","misunderstanding","railroad","moonlight","football"]
print("{0:20}{1:20}{2:20}".format("Word","Porter Stemmer","lancaster Stemmer"))
print("-"*60)
for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,ps.stem(word),lan.stem(word)))
'''

'\n# example found\nword_list = ["friend", "friendship", "friends", "friendships","stabil","destabilize","misunderstanding","railroad","moonlight","football"]\nprint("{0:20}{1:20}{2:20}".format("Word","Porter Stemmer","lancaster Stemmer"))\nprint("-"*60)\nfor word in word_list:\n    print("{0:20}{1:20}{2:20}".format(word,ps.stem(word),lan.stem(word)))\n'

I will create Porter stemming for the single column F of Task1 (data_cF_clean)

In [85]:
# create the pandas dataframe for Porter stemmer
data_cF_stem = pd.DataFrame(data_cF_clean, columns = ['Column F'])

data_cF_stem['Porter_stemmed'] = data_cF_stem['Column F'].apply(lambda x: [ps.stem(y) for y in x])
data_cF_stem.head(10)

,Column F,Porter_stemmed
0,[farming],[farm]
1,[ranching],[ranch]
2,[dairy],[dairi]
3,[fishery],[fisheri]
4,"[mining, metals]","[mine, metal]"
5,"[oil, energy]","[oil, energi]"
6,"[defense, space]","[defens, space]"
7,[pharmaceuticals],[pharmaceut]
8,"[food, production]","[food, product]"
9,"[aviation, aerospace]","[aviat, aerospac]"


Now I will create another column for Lancaster stemming

In [86]:
data_cF_stem['Lancaster_stemmed'] = data_cF_stem['Column F'].apply(lambda x: [lan.stem(y) for y in x])
data_cF_stem.head(10)

,Column F,Porter_stemmed,Lancaster_stemmed
0,[farming],[farm],[farm]
1,[ranching],[ranch],[ranch]
2,[dairy],[dairi],[dairy]
3,[fishery],[fisheri],[fishery]
4,"[mining, metals]","[mine, metal]","[min, met]"
5,"[oil, energy]","[oil, energi]","[oil, energy]"
6,"[defense, space]","[defens, space]","[defens, spac]"
7,[pharmaceuticals],[pharmaceut],[pharmaceut]
8,"[food, production]","[food, product]","[food, produc]"
9,"[aviation, aerospace]","[aviat, aerospac]","[avy, aerospac]"


The variable **data_cF_stem** has the original column, Porter stemmed and Lancaster stemmed. However, I will create a **data_cF_ps_stem** and its excel file **ColumnF_PorterStem.xlsx** just for Porter.

In [87]:
# remove 'Column F' and keep just 'stemmed'
data_cF_ps_stem = data_cF_stem.drop(columns=['Column F', 'Lancaster_stemmed'])

In [88]:
data_cF_ps_stem.head(5)

,Porter_stemmed
0,[farm]
1,[ranch]
2,[dairi]
3,[fisheri]
4,"[mine, metal]"


I will create a variable **data_cF_ps_stem2**, this variable will have the stemmed concatenated words of each row.

In [89]:
data_cF_ps_stem2 = data_cF_ps_stem.squeeze()

print(data_cF_ps_stem2)
type(data_cF_ps_stem2)

0                           [farm]
1                          [ranch]
2                          [dairi]
3                        [fisheri]
4                    [mine, metal]
                  ...             
139     [individu, famili, servic]
140             [religi, institut]
141         [civic, social, organ]
142    [non, profit, organ, manag]
143       [intern, trade, develop]
Name: Porter_stemmed, Length: 144, dtype: object


pandas.core.series.Series

In [90]:
data_cF_ps_stem2 = [' '.join(tup) for tup in data_cF_ps_stem2]
data_cF_ps_stem2

['farm',
 'ranch',
 'dairi',
 'fisheri',
 'mine metal',
 'oil energi',
 'defens space',
 'pharmaceut',
 'food product',
 'aviat aerospac',
 'automot',
 'chemic',
 'machineri',
 'shipbuild',
 'textil',
 'paper forest product',
 'railroad manufactur',
 'print',
 'electr electron manufactur',
 'plastic',
 'renew environ',
 'glass ceram concret',
 'packag contain',
 'industri autom',
 'util',
 'construct',
 'build materi',
 'civil engin',
 'cosmet',
 'apparel fashion',
 'sport good',
 'tobacco',
 'supermarket',
 'consum electron',
 'consum good',
 'furnitur',
 'retail',
 'food beverag',
 'consum servic',
 'wholesal',
 'wine spirit',
 'luxuri good jewelri',
 'packag freight deliveri',
 'transport truck railroad',
 'wareh',
 'airlin aviat',
 'maritim',
 'logist suppli chain',
 'import export',
 'hospit',
 'restaur',
 'comput hardwar',
 'comput softwar',
 'comput network',
 'internet',
 'semiconductor',
 'telecommun',
 'motion pictur film',
 'broadcast media',
 'newspap',
 'publish',
 'inform

In [91]:
data_cF_ps_stem2 = pd.DataFrame(data_cF_ps_stem2, columns = ['Porter'])
data_cF_ps_stem2.head()

,Porter
0,farm
1,ranch
2,dairi
3,fisheri
4,mine metal


To keep an excel file **ColumnF_PorterStem.xlsx**

In [92]:
# save an excel file
data_cF_ps_stem2.to_excel('ColumnF_PorterStem.xlsx', sheet_name = 'Porter', index= False, freeze_panes=(1,1))

I will create a **data_cF_lan_stem** and its excel file **ColumnF_LancasterStem.xlsx** just for Lancaster.

In [93]:
# remove 'Column F' and keep just 'stemmed'
data_cF_lan_stem = data_cF_stem.drop(columns=['Column F', 'Porter_stemmed'])

data_cF_lan_stem.head(5)

,Lancaster_stemmed
0,[farm]
1,[ranch]
2,[dairy]
3,[fishery]
4,"[min, met]"


I will create a variable **data_cF_lan_stem2**, this variable will have the stemmed concatenated words of each row.

In [94]:
data_cF_lan_stem2 = data_cF_lan_stem.squeeze()

print(data_cF_lan_stem2)
type(data_cF_lan_stem2)

0                       [farm]
1                      [ranch]
2                      [dairy]
3                    [fishery]
4                   [min, met]
                ...           
139      [individ, famy, serv]
140         [religy, institut]
141            [civ, soc, org]
142    [non, profit, org, man]
143    [intern, trad, develop]
Name: Lancaster_stemmed, Length: 144, dtype: object


pandas.core.series.Series

In [95]:
data_cF_lan_stem2 = [' '.join(tup) for tup in data_cF_lan_stem2]
data_cF_lan_stem2

['farm',
 'ranch',
 'dairy',
 'fishery',
 'min met',
 'oil energy',
 'defens spac',
 'pharmaceut',
 'food produc',
 'avy aerospac',
 'automot',
 'chem',
 'machinery',
 'shipbuild',
 'textil',
 'pap forest produc',
 'railroad manufact',
 'print',
 'elect electron manufact',
 'plast',
 'renew environ',
 'glass ceram concret',
 'pack contain',
 'indust autom',
 'util',
 'construct',
 'build mat',
 'civil engin',
 'cosmet',
 'apparel fash',
 'sport good',
 'tobacco',
 'supermarket',
 'consum electron',
 'consum good',
 'furnit',
 'retail',
 'food bev',
 'consum serv',
 'wholes',
 'win spirit',
 'luxury good jewelry',
 'pack freight delivery',
 'transport truck railroad',
 'wareh',
 'airlin avy',
 'maritim',
 'log supply chain',
 'import export',
 'hospit',
 'resta',
 'comput hardw',
 'comput softw',
 'comput network',
 'internet',
 'semiconduc',
 'telecommun',
 'mot pict film',
 'broadcast med',
 'newspap',
 'publ',
 'inform technolog serv',
 'writ edit',
 'comput gam',
 'onlin med',
 'com

In [96]:
data_cF_lan_stem2 = pd.DataFrame(data_cF_lan_stem2, columns = ['Lancaster'])
data_cF_lan_stem2.head()

,Lancaster
0,farm
1,ranch
2,dairy
3,fishery
4,min met


To keep an excel file **ColumnF_LancasterStem.xlsx**

In [97]:
# save an excel file
data_cF_lan_stem2.to_excel('ColumnF_LancasterStem.xlsx', sheet_name = 'Lancaster', index= False, freeze_panes=(1,1))

I will create new stemming dataset for data_cF_cD_clean dataframe. For assessing a new matching case.

In [98]:
# Stemming on merged ColF and ColD dataframe, remember the dataframe should come in list form for stemming
# Create the pandas dataframe for Porter stemmer and Lancaster stemmer

data_cF_cD_stem =  pd.DataFrame(data_cF_cD_clean, columns= ['ColF ColD'])
data_cF_cD_stem['Porter_stemmed'] = data_cF_cD_stem['ColF ColD'].apply(lambda x: [ps.stem(y) for y in x])
data_cF_cD_stem['Lancaster_stemmed'] = data_cF_cD_stem['ColF ColD'].apply(lambda x: [lan.stem(y) for y in x])
data_cF_cD_stem.head(20)

,ColF ColD,Porter_stemmed,Lancaster_stemmed
0,"[farming, crop, animal, production, hunting, related, service, activities]","[farm, crop, anim, product, hunt, relat, servic, activ]","[farm, crop, anim, produc, hunt, rel, serv, act]"
1,"[ranching, crop, animal, production, hunting, related, service, activities]","[ranch, crop, anim, product, hunt, relat, servic, activ]","[ranch, crop, anim, produc, hunt, rel, serv, act]"
2,"[dairy, crop, animal, production, hunting, related, service, activities]","[dairi, crop, anim, product, hunt, relat, servic, activ]","[dairy, crop, anim, produc, hunt, rel, serv, act]"
3,"[fishery, fishing, aquaculture]","[fisheri, fish, aquacultur]","[fishery, fish, aquacult]"
4,"[mining, metals, coal, lignite]","[mine, metal, coal, lignit]","[min, met, coal, lignit]"
5,"[oil, energy, extraction, crude, petroleum, natural, gas]","[oil, energi, extract, crude, petroleum, natur, ga]","[oil, energy, extract, crud, petrole, nat, gas]"
6,"[defense, space, manufacture, fabricated, metal, products, except, machinery, equip]","[defens, space, manufactur, fabric, metal, product, except, machineri, equip]","[defens, spac, manufact, fabr, met, produc, exceiv, machinery, equip]"
7,"[pharmaceuticals, manufacture, medicinal, chemical, botanical, products]","[pharmaceut, manufactur, medicin, chemic, botan, product]","[pharmaceut, manufact, medicin, chem, bot, produc]"
8,"[food, production, manufacture, products]","[food, product, manufactur, product]","[food, produc, manufact, produc]"
9,"[aviation, aerospace, manufacture, transport, equipment]","[aviat, aerospac, manufactur, transport, equip]","[avy, aerospac, manufact, transport, equip]"


In [99]:
# remove columns for converting to series
data_cF_cD_ps_stem = data_cF_cD_stem.drop(columns = ['ColF ColD', 'Lancaster_stemmed'])
data_cF_cD_lan_stem = data_cF_cD_stem.drop(columns =['ColF ColD', 'Porter_stemmed'])
data_cF_cD_ps_stem.head(5), data_cF_cD_lan_stem.head(5)

(                                             Porter_stemmed
 0   [farm, crop, anim, product, hunt, relat, servic, activ]
 1  [ranch, crop, anim, product, hunt, relat, servic, activ]
 2  [dairi, crop, anim, product, hunt, relat, servic, activ]
 3                               [fisheri, fish, aquacultur]
 4                               [mine, metal, coal, lignit],
                                    Lancaster_stemmed
 0   [farm, crop, anim, produc, hunt, rel, serv, act]
 1  [ranch, crop, anim, produc, hunt, rel, serv, act]
 2  [dairy, crop, anim, produc, hunt, rel, serv, act]
 3                          [fishery, fish, aquacult]
 4                           [min, met, coal, lignit])

In [100]:
data_cF_cD_ps_stem2 = data_cF_cD_ps_stem.squeeze()
data_cF_cD_lan_stem2 = data_cF_cD_lan_stem.squeeze()

data_cF_cD_ps_stem2, data_cF_cD_lan_stem2

(0                [farm, crop, anim, product, hunt, relat, servic, activ]
 1               [ranch, crop, anim, product, hunt, relat, servic, activ]
 2               [dairi, crop, anim, product, hunt, relat, servic, activ]
 3                                            [fisheri, fish, aquacultur]
 4                                            [mine, metal, coal, lignit]
                                      ...                                
 139                    [individu, famili, servic, person, servic, activ]
 140             [religi, institut, activ, profession, membership, organ]
 141         [civic, social, organ, activ, profession, membership, organ]
 142    [non, profit, organ, manag, activ, profession, membership, organ]
 143       [intern, trade, develop, activ, profession, membership, organ]
 Name: Porter_stemmed, Length: 144, dtype: object,
 0       [farm, crop, anim, produc, hunt, rel, serv, act]
 1      [ranch, crop, anim, produc, hunt, rel, serv, act]
 2      [dairy, cro

In [101]:
# concatenate
data_cF_cD_ps_stem2 = [' '.join(tup) for tup in data_cF_cD_ps_stem2]
data_cF_cD_lan_stem2 = [' '.join(tup) for tup in data_cF_cD_lan_stem2]

data_cF_cD_ps_stem2,  data_cF_cD_lan_stem2

(['farm crop anim product hunt relat servic activ',
  'ranch crop anim product hunt relat servic activ',
  'dairi crop anim product hunt relat servic activ',
  'fisheri fish aquacultur',
  'mine metal coal lignit',
  'oil energi extract crude petroleum natur ga',
  'defens space manufactur fabric metal product except machineri equip',
  'pharmaceut manufactur medicin chemic botan product',
  'food product manufactur product',
  'aviat aerospac manufactur transport equip',
  'automot manufactur motor vehicl trailer semi',
  'chemic manufactur chemic product',
  'machineri manufactur equip n e c',
  'shipbuild manufactur transport equip',
  'textil manufactur',
  'paper forest product manufactur',
  'railroad manufactur transport equip',
  'print reproduct record media',
  'electr electron manufactur manufactur comput optic product',
  'plastic manufactur rubber product',
  'renew environ manufactur',
  'glass ceram concret manufactur',
  'packag contain manufactur',
  'industri autom ma

In [102]:
# convert to dataFrame
data_cF_cD_ps_stem2 = pd.DataFrame(data_cF_cD_ps_stem2, columns = ['Porter'])
data_cF_cD_lan_stem2 = pd.DataFrame(data_cF_cD_lan_stem2, columns = ['Lancaster'])

data_cF_cD_ps_stem2, data_cF_cD_lan_stem2

(                                                       Porter
 0              farm crop anim product hunt relat servic activ
 1             ranch crop anim product hunt relat servic activ
 2             dairi crop anim product hunt relat servic activ
 3                                     fisheri fish aquacultur
 4                                      mine metal coal lignit
 ..                                                        ...
 139                individu famili servic person servic activ
 140         religi institut activ profession membership organ
 141      civic social organ activ profession membership organ
 142  non profit organ manag activ profession membership organ
 143    intern trade develop activ profession membership organ
 
 [144 rows x 1 columns],
                                     Lancaster
 0     farm crop anim produc hunt rel serv act
 1    ranch crop anim produc hunt rel serv act
 2    dairy crop anim produc hunt rel serv act
 3                       fish

In [103]:
# save excel file
data_cF_cD_ps_stem2.to_excel('ColF and ColD_PorterStem.xlsx', sheet_name= 'Porter', index= False, freeze_panes=(1,1))
data_cF_cD_lan_stem2.to_excel('ColF and ColD_LancasterStem.xlsx', sheet_name= 'Lancaster', index= False, freeze_panes=(1,1))

---------------- FULL TEXT TASK 1 ----------------------

Doing the same for the **data_clean** of TASK1 for Porter and Lancaster.

In [104]:
# create the pandas dataframe for Porter stemmer
data_ps_stem = pd.DataFrame(data_clean, columns = ['Full_text_TASK1'])

# to see the full column  (instruction above, no needed for this anymore)                        
# pd.set_option('display.max_colwidth', None) 


data_ps_stem['Porter_stemmed'] = data_ps_stem['Full_text_TASK1'].apply(lambda x: [ps.stem(y) for y in x])
data_ps_stem.head(10)

,Full_text_TASK1,Porter_stemmed
0,"[farming, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[farm, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
1,"[ranching, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[ranch, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
2,"[dairy, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[dairi, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
3,"[fishery, agriculture, forestry, fishing, aquaculture]","[fisheri, agricultur, forestri, fish, aquacultur]"
4,"[mining, metals, energy, b, quarrying, coal, lignite]","[mine, metal, energi, b, quarri, coal, lignit]"
5,"[oil, energy, mining, b, quarrying, extraction, crude, petroleum, natural, gas]","[oil, energi, mine, b, quarri, extract, crude, petroleum, natur, ga]"
6,"[defense, space, manufacturing, c, manufacture, fabricated, metal, products, except, machinery, equip]","[defens, space, manufactur, c, manufactur, fabric, metal, product, except, machineri, equip]"
7,"[pharmaceuticals, healthcare, c, manufacturing, manufacture, medicinal, chemical, botanical, products]","[pharmaceut, healthcar, c, manufactur, manufactur, medicin, chemic, botan, product]"
8,"[food, production, manufacturing, c, manufacture, products]","[food, product, manufactur, c, manufactur, product]"
9,"[aviation, aerospace, manufacturing, c, manufacture, transport, equipment]","[aviat, aerospac, manufactur, c, manufactur, transport, equip]"


I will keep data_ps_stem just with the Porter_stemmed column

In [105]:
# remove 'Full_text_TASK1' and keep just 'stemmed'
data_ps_stem = data_ps_stem.drop(columns=['Full_text_TASK1'])

In [106]:
data_ps_stem.head(5)

,Porter_stemmed
0,"[farm, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
1,"[ranch, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
2,"[dairi, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]"
3,"[fisheri, agricultur, forestri, fish, aquacultur]"
4,"[mine, metal, energi, b, quarri, coal, lignit]"


I will create a new variable **data_ps_stem2** equal to the **data_ps_stem** and it will converted to series, the words will be concatenated per row, and again converted to dataframe. Finally, an excel file will be created.

In [107]:
data_ps_stem2 = data_ps_stem.squeeze()
data_ps_stem2

0       [farm, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]
1      [ranch, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]
2      [dairi, agricultur, forestri, fish, crop, anim, product, hunt, relat, servic, activ]
3                                         [fisheri, agricultur, forestri, fish, aquacultur]
4                                            [mine, metal, energi, b, quarri, coal, lignit]
                                               ...                                         
139                            [individu, famili, servic, nonprofit, servic, activ, person]
140             [religi, institut, nonprofit, servic, activ, profession, membership, organ]
141         [civic, social, organ, nonprofit, servic, activ, profession, membership, organ]
142    [non, profit, organ, manag, nonprofit, servic, activ, profession, membership, organ]
143       [intern, trade, develop, nonprofit, servic, activ, profession, members

In [108]:
type(data_ps_stem2)

pandas.core.series.Series

In [109]:
data_ps_stem2 = [' '.join(tup) for tup in data_ps_stem2]
data_ps_stem2

['farm agricultur forestri fish crop anim product hunt relat servic activ',
 'ranch agricultur forestri fish crop anim product hunt relat servic activ',
 'dairi agricultur forestri fish crop anim product hunt relat servic activ',
 'fisheri agricultur forestri fish aquacultur',
 'mine metal energi b quarri coal lignit',
 'oil energi mine b quarri extract crude petroleum natur ga',
 'defens space manufactur c manufactur fabric metal product except machineri equip',
 'pharmaceut healthcar c manufactur manufactur medicin chemic botan product',
 'food product manufactur c manufactur product',
 'aviat aerospac manufactur c manufactur transport equip',
 'automot manufactur c manufactur motor vehicl trailer semi',
 'chemic manufactur c manufactur chemic product',
 'machineri manufactur c manufactur equip n e',
 'shipbuild manufactur c manufactur transport equip',
 'textil manufactur c manufactur',
 'paper forest product manufactur c manufactur',
 'railroad manufactur manufactur c transport equ

In [110]:
data_ps_stem2 = pd.DataFrame(data_ps_stem2, columns = ['Porter'])
data_ps_stem2.head()

,Porter
0,farm agricultur forestri fish crop anim product hunt relat servic activ
1,ranch agricultur forestri fish crop anim product hunt relat servic activ
2,dairi agricultur forestri fish crop anim product hunt relat servic activ
3,fisheri agricultur forestri fish aquacultur
4,mine metal energi b quarri coal lignit


To keep an excel file **TASK1_PorterStem.xlsx**

In [111]:
# save an excel file
data_ps_stem2.to_excel('TASK1_PorterStem.xlsx', sheet_name = 'Porter', index= False, freeze_panes=(1,1))

Now, I will do the same with the **data_clean** for **Lancaster**.

In [112]:
# create the pandas dataframe for Lancaster stemmer
data_lan_stem = pd.DataFrame(data_clean, columns = ['Full_text_TASK1'])

data_lan_stem['Lancaster_stemmed'] = data_lan_stem['Full_text_TASK1'].apply(lambda x: [lan.stem(y) for y in x])
data_lan_stem.head(10)


,Full_text_TASK1,Lancaster_stemmed
0,"[farming, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[farm, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
1,"[ranching, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[ranch, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
2,"[dairy, agriculture, forestry, fishing, crop, animal, production, hunting, related, service, activities]","[dairy, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
3,"[fishery, agriculture, forestry, fishing, aquaculture]","[fishery, agricult, forestry, fish, aquacult]"
4,"[mining, metals, energy, b, quarrying, coal, lignite]","[min, met, energy, b, quarry, coal, lignit]"
5,"[oil, energy, mining, b, quarrying, extraction, crude, petroleum, natural, gas]","[oil, energy, min, b, quarry, extract, crud, petrole, nat, gas]"
6,"[defense, space, manufacturing, c, manufacture, fabricated, metal, products, except, machinery, equip]","[defens, spac, manufact, c, manufact, fabr, met, produc, exceiv, machinery, equip]"
7,"[pharmaceuticals, healthcare, c, manufacturing, manufacture, medicinal, chemical, botanical, products]","[pharmaceut, healthc, c, manufact, manufact, medicin, chem, bot, produc]"
8,"[food, production, manufacturing, c, manufacture, products]","[food, produc, manufact, c, manufact, produc]"
9,"[aviation, aerospace, manufacturing, c, manufacture, transport, equipment]","[avy, aerospac, manufact, c, manufact, transport, equip]"


In [113]:
# remove 'Full_text_TASK1' and keep just 'stemmed'
data_lan_stem = data_lan_stem.drop(columns=['Full_text_TASK1'])

In [114]:
data_lan_stem.head(5)

,Lancaster_stemmed
0,"[farm, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
1,"[ranch, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
2,"[dairy, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]"
3,"[fishery, agricult, forestry, fish, aquacult]"
4,"[min, met, energy, b, quarry, coal, lignit]"


I will create a new variable **data_lan_stem2** equal to the **data_lan_stem** and it will converted to series, the words will be concatenated per row, and again converted to dataframe. Finally, an excel file will be created.

In [115]:
data_lan_stem2 = data_lan_stem.squeeze()
data_lan_stem2

0       [farm, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]
1      [ranch, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]
2      [dairy, agricult, forestry, fish, crop, anim, produc, hunt, rel, serv, act]
3                                    [fishery, agricult, forestry, fish, aquacult]
4                                      [min, met, energy, b, quarry, coal, lignit]
                                          ...                                     
139                            [individ, famy, serv, nonprofit, serv, act, person]
140                   [religy, institut, nonprofit, serv, act, profess, memb, org]
141                      [civ, soc, org, nonprofit, serv, act, profess, memb, org]
142              [non, profit, org, man, nonprofit, serv, act, profess, memb, org]
143              [intern, trad, develop, nonprofit, serv, act, profess, memb, org]
Name: Lancaster_stemmed, Length: 144, dtype: object

In [116]:
type(data_lan_stem2)

pandas.core.series.Series

In [117]:
data_lan_stem2 = [' '.join(tup) for tup in data_lan_stem2]
data_lan_stem2

['farm agricult forestry fish crop anim produc hunt rel serv act',
 'ranch agricult forestry fish crop anim produc hunt rel serv act',
 'dairy agricult forestry fish crop anim produc hunt rel serv act',
 'fishery agricult forestry fish aquacult',
 'min met energy b quarry coal lignit',
 'oil energy min b quarry extract crud petrole nat gas',
 'defens spac manufact c manufact fabr met produc exceiv machinery equip',
 'pharmaceut healthc c manufact manufact medicin chem bot produc',
 'food produc manufact c manufact produc',
 'avy aerospac manufact c manufact transport equip',
 'automot manufact c manufact mot vehic trail sem',
 'chem manufact c manufact chem produc',
 'machinery manufact c manufact equip n e',
 'shipbuild manufact c manufact transport equip',
 'textil manufact c manufact',
 'pap forest produc manufact c manufact',
 'railroad manufact manufact c transport equip',
 'print med commun c manufact reproduc record',
 'elect electron manufact c manufact comput opt produc',
 'pl

In [118]:
data_lan_stem2 = pd.DataFrame(data_lan_stem2, columns = ['Lancaster'])
data_lan_stem2.head()

,Lancaster
0,farm agricult forestry fish crop anim produc hunt rel serv act
1,ranch agricult forestry fish crop anim produc hunt rel serv act
2,dairy agricult forestry fish crop anim produc hunt rel serv act
3,fishery agricult forestry fish aquacult
4,min met energy b quarry coal lignit


To keep an excel file **TASK1_LancasterStem.xlsx**

In [119]:
# save an excel file
data_lan_stem2.to_excel('TASK1_LancasterStem.xlsx', sheet_name = 'Lancaster', index= False, freeze_panes=(1,1))

--------------- NOW WORKING WITH **IAB** -------------------

Now, I will do the same **Porter stemmer** on **IAB**.

In [120]:
# create the pandas dataframe for Porter stemmer in IAB
data_IAB_ps_stem = pd.DataFrame(data_IAB_clean, columns = ['Text_IAB'])

In [121]:
data_IAB_ps_stem['Porter_stemmed'] = data_IAB_ps_stem['Text_IAB'].apply(lambda x: [ps.stem(y) for y in x])
data_IAB_ps_stem.head(10)

,Text_IAB,Porter_stemmed
0,[automotive],[automot]
1,"[auto, body, styles, automotive]","[auto, bodi, style, automot]"
2,"[commercial, trucks, automotive, auto, body, styles]","[commerci, truck, automot, auto, bodi, style]"
3,"[sedan, automotive, auto, body, styles]","[sedan, automot, auto, bodi, style]"
4,"[station, wagon, automotive, auto, body, styles]","[station, wagon, automot, auto, bodi, style]"
5,"[suv, automotive, auto, body, styles]","[suv, automot, auto, bodi, style]"
6,"[van, automotive, auto, body, styles]","[van, automot, auto, bodi, style]"
7,"[convertible, automotive, auto, body, styles]","[convert, automot, auto, bodi, style]"
8,"[coupe, automotive, auto, body, styles]","[coup, automot, auto, bodi, style]"
9,"[crossover, automotive, auto, body, styles]","[crossov, automot, auto, bodi, style]"


In [122]:
# remove 'Full_text_TASK1' and keep just 'stemmed'   JuST run this one
data_IAB_ps_stem = data_IAB_ps_stem.drop(columns=['Text_IAB'])

In [123]:
data_IAB_ps_stem.head(10)

,Porter_stemmed
0,[automot]
1,"[auto, bodi, style, automot]"
2,"[commerci, truck, automot, auto, bodi, style]"
3,"[sedan, automot, auto, bodi, style]"
4,"[station, wagon, automot, auto, bodi, style]"
5,"[suv, automot, auto, bodi, style]"
6,"[van, automot, auto, bodi, style]"
7,"[convert, automot, auto, bodi, style]"
8,"[coup, automot, auto, bodi, style]"
9,"[crossov, automot, auto, bodi, style]"


To do the work around to concantenate words in each row

In [124]:
data_IAB_ps_stem2 = data_IAB_ps_stem.squeeze()
data_IAB_ps_stem2

0                                           [automot]
1                        [auto, bodi, style, automot]
2       [commerci, truck, automot, auto, bodi, style]
3                 [sedan, automot, auto, bodi, style]
4        [station, wagon, automot, auto, bodi, style]
                            ...                      
686            [adult, video, game, game, game, genr]
687    [casino, gambl, video, game, game, game, genr]
688           [famili, video, game, game, game, genr]
689           [horror, video, game, game, game, genr]
690             [race, video, game, game, game, genr]
Name: Porter_stemmed, Length: 691, dtype: object

In [125]:
data_IAB_ps_stem2 = [' '.join(tup) for tup in data_IAB_ps_stem2]
data_IAB_ps_stem2

['automot',
 'auto bodi style automot',
 'commerci truck automot auto bodi style',
 'sedan automot auto bodi style',
 'station wagon automot auto bodi style',
 'suv automot auto bodi style',
 'van automot auto bodi style',
 'convert automot auto bodi style',
 'coup automot auto bodi style',
 'crossov automot auto bodi style',
 'hatchback automot auto bodi style',
 'microcar automot auto bodi style',
 'minivan automot auto bodi style',
 'road vehicl automot auto bodi style',
 'pickup truck automot auto bodi style',
 'auto type automot',
 'budget car automot auto type',
 'certifi pre own car automot auto type',
 'classic car automot auto type',
 'concept car automot auto type',
 'driverless car automot auto type',
 'green vehicl automot auto type',
 'luxuri car automot auto type',
 'perform car automot auto type',
 'car cultur automot',
 'dash cam video automot',
 'motorcycl automot',
 'road side assist automot',
 'scooter automot',
 'auto buy sell automot',
 'auto insur automot',
 'auto

In [126]:
data_IAB_ps_stem2 = pd.DataFrame(data_IAB_ps_stem2, columns =['Porter'])
data_IAB_ps_stem2.head()

,Porter
0,automot
1,auto bodi style automot
2,commerci truck automot auto bodi style
3,sedan automot auto bodi style
4,station wagon automot auto bodi style


To create an excel file **IAB_PorterStem.xlsx**

In [127]:
# save an excel file
data_IAB_ps_stem2.to_excel('IAB_PorterStem.xlsx', sheet_name = 'Porter', index= False, freeze_panes=(1,1))

Now, I will do the same **Lancaster stemmer** on **IAB**.

In [128]:
# create the pandas dataframe for Lancaster stemmer
data_IAB_lan_stem = pd.DataFrame(data_IAB_clean, columns = ['Text_IAB'])

In [129]:
data_IAB_lan_stem['Lancaster_stemmed'] = data_IAB_lan_stem['Text_IAB'].apply(lambda x: [lan.stem(y) for y in x])
data_IAB_lan_stem.head(10)

,Text_IAB,Lancaster_stemmed
0,[automotive],[automot]
1,"[auto, body, styles, automotive]","[auto, body, styl, automot]"
2,"[commercial, trucks, automotive, auto, body, styles]","[commerc, truck, automot, auto, body, styl]"
3,"[sedan, automotive, auto, body, styles]","[sed, automot, auto, body, styl]"
4,"[station, wagon, automotive, auto, body, styles]","[stat, wagon, automot, auto, body, styl]"
5,"[suv, automotive, auto, body, styles]","[suv, automot, auto, body, styl]"
6,"[van, automotive, auto, body, styles]","[van, automot, auto, body, styl]"
7,"[convertible, automotive, auto, body, styles]","[convert, automot, auto, body, styl]"
8,"[coupe, automotive, auto, body, styles]","[coup, automot, auto, body, styl]"
9,"[crossover, automotive, auto, body, styles]","[crossov, automot, auto, body, styl]"


In [130]:
# remove 'Full_text_TASK1' and keep just 'stemmed'
data_IAB_lan_stem = data_IAB_lan_stem.drop(columns=['Text_IAB'])
data_IAB_lan_stem.head(10)

,Lancaster_stemmed
0,[automot]
1,"[auto, body, styl, automot]"
2,"[commerc, truck, automot, auto, body, styl]"
3,"[sed, automot, auto, body, styl]"
4,"[stat, wagon, automot, auto, body, styl]"
5,"[suv, automot, auto, body, styl]"
6,"[van, automot, auto, body, styl]"
7,"[convert, automot, auto, body, styl]"
8,"[coup, automot, auto, body, styl]"
9,"[crossov, automot, auto, body, styl]"


In [131]:
data_IAB_lan_stem2 = data_IAB_lan_stem.squeeze()
data_IAB_lan_stem2

0                                        [automot]
1                      [auto, body, styl, automot]
2      [commerc, truck, automot, auto, body, styl]
3                 [sed, automot, auto, body, styl]
4         [stat, wagon, automot, auto, body, styl]
                          ...                     
686            [adult, video, gam, gam, gam, genr]
687    [casino, gambl, video, gam, gam, gam, genr]
688             [famy, video, gam, gam, gam, genr]
689              [hor, video, gam, gam, gam, genr]
690              [rac, video, gam, gam, gam, genr]
Name: Lancaster_stemmed, Length: 691, dtype: object

In [132]:
data_IAB_lan_stem2 = [' '.join(tup) for tup in data_IAB_lan_stem2]
data_IAB_lan_stem2

['automot',
 'auto body styl automot',
 'commerc truck automot auto body styl',
 'sed automot auto body styl',
 'stat wagon automot auto body styl',
 'suv automot auto body styl',
 'van automot auto body styl',
 'convert automot auto body styl',
 'coup automot auto body styl',
 'crossov automot auto body styl',
 'hatchback automot auto body styl',
 'microc automot auto body styl',
 'min automot auto body styl',
 'road vehic automot auto body styl',
 'pickup truck automot auto body styl',
 'auto typ automot',
 'budget car automot auto typ',
 'cert pre own car automot auto typ',
 'class car automot auto typ',
 'conceiv car automot auto typ',
 'driverless car automot auto typ',
 'green vehic automot auto typ',
 'luxury car automot auto typ',
 'perform car automot auto typ',
 'car cult automot',
 'dash cam video automot',
 'motorcyc automot',
 'road sid assist automot',
 'scoot automot',
 'auto buy sel automot',
 'auto ins automot',
 'auto part automot',
 'auto recal automot',
 'auto repai

In [133]:
data_IAB_lan_stem2 = pd.DataFrame(data_IAB_lan_stem2, columns = ['Lancaster'])
data_IAB_lan_stem2.head()

,Lancaster
0,automot
1,auto body styl automot
2,commerc truck automot auto body styl
3,sed automot auto body styl
4,stat wagon automot auto body styl


To create an excel file **IAB_LancasterStem.xlsx**

In [134]:
# save an excel file
data_IAB_lan_stem2.to_excel('IAB_LancasterStem.xlsx', sheet_name = 'Lancaster', index= False, freeze_panes=(1,1))

In [135]:
IAB_codes['IAB_Porter'] = data_IAB_ps_stem2['Porter']
IAB_codes.head()

,Name,Unique ID,IAB_clean,IAB_Porter
0,Automotive,1,automotive,automot
1,Auto Body Styles,2,auto body styles automotive,auto bodi style automot
2,Commercial Trucks,3,commercial trucks automotive auto body styles,commerci truck automot auto bodi style
3,Sedan,4,sedan automotive auto body styles,sedan automot auto bodi style
4,Station Wagon,5,station wagon automotive auto body styles,station wagon automot auto bodi style


In [136]:
IAB_codes['IAB_Lancaster'] = data_IAB_lan_stem2['Lancaster']
IAB_codes.head()

,Name,Unique ID,IAB_clean,IAB_Porter,IAB_Lancaster
0,Automotive,1,automotive,automot,automot
1,Auto Body Styles,2,auto body styles automotive,auto bodi style automot,auto body styl automot
2,Commercial Trucks,3,commercial trucks automotive auto body styles,commerci truck automot auto bodi style,commerc truck automot auto body styl
3,Sedan,4,sedan automotive auto body styles,sedan automot auto bodi style,sed automot auto body styl
4,Station Wagon,5,station wagon automotive auto body styles,station wagon automot auto bodi style,stat wagon automot auto body styl


So far the following variables and files have been created:

- **data_cF_stem** is a dataframe of the columnF TASK1
- **data_cF_ps_stem** is a dataframe of the column F TASK1 after the Porter stemming (no words concatenated yet) 
- **data_cF_ps_stem2** is a dataframe of Column F TASK1 with concatenated word after the Porter stemming and its excel file **ColumnF_PorterStem.xlsx**

- **data_cF_lan_stem** is a dataframe of the column F TASK1 after the Lancaster stemming (no words concatenated yet) 
- **data_cF_lan_stem2** is a dataframe of Column F TASK1 with concatenated word after the Lancaster stemming and its excel file **ColumnF_LancasterStem.xlsx**

- **data_cF_cD_ps_stem** is a dataframe of the columns F and D TASK1 after the Porter stemming (no words concatenated yet)
- **data_cF_cD_ps_stem2** is a datafram of the Columns F and D TASK1 with concatenated words after Porter stemming and its excel file is **ColF and ColD_PorterStem.xlsx**

- **data_cF_cD_lan_stem** is a dataframe of the columns F and D TASK1 after the Lancaster stemming (no words concatenated yet)
- **data_cF_cD_lan_stem2** is a datafram of the Columns F and D TASK1 with concatenated words after Lancaster stemming and its excel file is **ColF and ColD_LancasterStem.xlsx**

- **data_ps_stem** is a data frame of the joined columns of TASK1 (clean) after Porter stemming (no words concatenated yet)
- **data_ps_stem2** is a dataframe of the joined columns of TASK1 with concatenated word after the Porter stemming and its excel file **TASK1_PorterStem.xlsx**

- **data_lan_stem** is a data frame of the joined columns of TASK1 (clean) after Lancaster
- **data_lan_stem2** is a dataframe of the joined columns of TASK1 with concatenated word after the Lancaster stemming and its excel file **TASK1_LancasterStem.xlsx**

- **data_IAB_ps_stem** is a data frame of the joined columns of IAB (clean) after Porter stemming (no words concatenated yet) 
- **data_IAB_ps_stem2** is a dataframe of the joined columns of IAB with concatenated words after Porter Stemming and and  excel file **IAB_PorterSem.xlsx**

- **data_IAB_lan_stem** is a data frame of the joined columns of IAB (clean) after Lancaster stemming (no words concatenated yet) 
- **data_IAB_lan_stem2** is a dataframe of the joined columns of IAB with concatenated words after Lancaster Stemming and and  excel file **IAB_LancasterSem.xlsx**

In [ ]:
def show_dataframes(sheet_name):
    df1 = "data['Full_text_TASK1']" + " merge_cells, remove punctuation, NO CONCATENATED words and NO REMOVE STOPWORDS"
    df2 = "data_clean2['Full_text_TASK1']" + " merge_cells, PREPROCESSED, and CONCATENATED words"
    df3 = "data_lan_stem['Lancaster_stemmed']" + " merge_cells, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df4 = "data_lan_stem2['Lancaster']" + " merge_cells, PREPROCESSED, STEMMED and CONCATENATED words"
    df5 = "data_ps_stem['Porter_stemmed']" + " merge_cells, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df6 = "data_ps_stem2['Porter']" + " merge_cells, PREPROCESSED, STEMMED and CONCATENATED words"
    
    df7 = "data_cF['Column F']" + " only ColumnF, remove punctuation, NO CONCATENATED words and NO REMOVE STOPWORDS"
    df8 = "data_cF_clean2['Column F']" + " only ColumnF, PREPROCESSED, and CONCATENATED words"
    df9 = "data_cF_lan_stem['Lancaster_stemmed']" + " only ColumnF, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df10 = "data_cF_lan_stem2['Lancaster']" + " only ColumnF, PREPROCESSED, STEMMED and CONCATENATED words"
    df11 = "data_cF_ps_stem['Porter_stemmed']" + " only ColumnF, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df12 = "data_cF_ps_stem2['Porter']" + " only ColumnF, PREPROCESSED, STEMMED and CONCATENATED words"
    
    df22 = "data_cF_cD_clean['ColF ColD']" + " merge_cells, remove punctuation, NO CONCATENATED words and NO REMOVE STOPWORDS"
    df23 = "data_cF_cD_clean2['ColF ColD']" + " merge_cells, PREPROCESSED, and CONCATENATED words"
    df24 = "data_cF_cD_ps_stem['Porter_stemmed']"+ " only Columns F and D, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df25 = "data_cF_cD_ps_stem2['Porter']" + " only Columns F and D, PREPROCESSED, STEMMED and CONCATENATED words"
    df26 = "data_cF_cD_lan_stem['Lancaster_stemmed']" + " only Columns F and D, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df27 = "data_cF_cD_lan_stem2['Lancaster']" + " only Columns F and D, PREPROCESSED, STEMMED and CONCATENATED words"
    
    df13 = "data_IAB['Text_IAB']" + " merge_cells, remove punctuation, NO CONCATENATED words and NO REMOVE STOPWORDS"
    df14 = "data_IAB_clean2['Text_IAB']" + " merge_cells, PREPROCESSED and CONCATENAED words"
    df15 = "data_IAB_lan_stem['Lancaster_stemmed']" + " merge_cells, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df16 = "data_IAB_lan_stem2['Lancaster']" + " merge_cells, PREPROCESSED, STEMMED and CONCATENATED words"
    df17 = "data_IAB_ps_stem['Porter_stemmed']" + " merge_cells, PREPROCESSED, STEMMED and NO CONCATENATED words"
    df18 = "data_IAB_ps_stem2['Porter']" + " merge_cells, PREPROCESSED, STEMMED and CONCATENATED words"
    
    df19 = "data_NACE['Text_NACE']" + " merge_cells, remove punctuation, NO CONCATENATED words and NO REMOVE STOPWORDS"
    df20 = "data_NACE_clean['Text_NACE']" + " merge_cells, PREPROCESSED, and NO CONCATENATED words"
    df21 = "data_NACE_clean2['Text_NACE']" + " merge_cells, PREPROCESSED, and CONCATENATED words"
    
    if sheet_name == 'TASK1':
        print ('NO CONCATENATED and NO REMOVE STOPWORDS \n')
        print(df1), print(df7), print(df22)
        print ('\n PREPROCESSED and NO CONCATENATED words \n')
        print(df3), print(df5), print(df9), print(df11), print(df24), print(df26)
        print ('\n PREPROCESSED and CONCATENATED words \n')
        print(df2), print(df4), print(df6), print(df8), print(df10), print(df12), print(df23), print(df25), print(df27)
    elif sheet_name == 'IAB':
        print ('NO CONCATENATED and NO REMOVE STOPWORDS \n')
        print(df13)
        print ('\n PREPROCESSED and NO CONCATENATED words \n')
        print(df15), print(df17)
        print ('\n PREPROCESSED and CONCATENATED words \n')
        print(df14), print(df16), print(df18)
    elif sheet_name == "NACE":
        print ('NO CONCATENATED and NO REMOVE STOPWORDS \n')
        print(df19)
        print ('\n PREPROCESSED and NO CONCATENATED words \n')
        print(df20)
        print ('\n PREPROCESSED and CONCATENATED words \n')
        print(df21)
    
   
show_dataframes('TASK1')

## Matching

In [137]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process, utils

Very simple ratio function. using fuzz ratio and fuzz partial ratio.

**Ratio (Simple Ratio)**
When you have a very simple set of strings which look almost similar with their words, you can use the simple ratio from the FuzzyWuzzy package.

**Partial Ratio**
when you wouldn’t like to take into consideration all the small details like stop words, punctuations, capital letters etc., it’s better to use the Partial Ratio. 
When dealing with substrings, i.e., one short string being a part of some other long string, we use the Partial Ratio function. The mechanism of this ratio deals with something known as ‘optimal partial logic’. 

In [138]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

The following function **Ratios** gives the ratio and partial ratio. 

**How to get the results in descendt order?**

In [139]:
def Ratios (word, threshold = 50):
    print('%8s %8s %15s %17s %-20s' % ('Ratio', 'Partial_R', 'location', 'description', 'group'))
    for i in range (2, sh_IAB.max_row + 1):
        Ratio = fuzz.ratio(word.lower(), sh_IAB.cell(row = i, column = 1).value.lower())
        Partial_R = fuzz.partial_ratio(word.lower(), sh_IAB.cell(row = i, column = 1).value.lower())
        if Ratio > threshold and Partial_R > threshold:
            location = sh_IAB.cell(row =i , column = 1)
            description = sh_IAB.cell(row = i, column = 1).value
            group = sh_IAB.cell(row = i, column = 4).value
            print('%8d %8d %15s %17s %-20s' % (Ratio, Partial_R, location, description, group))
                      

In [140]:
Ratios('auto')


   Ratio Partial_R        location       description group               
      57      100 <Cell 'IAB'.A2>        Automotive Automotive          
      62      100 <Cell 'IAB'.A17>         Auto Type Automotive          
      57      100 <Cell 'IAB'.A33>        Auto Parts Automotive          
      53      100 <Cell 'IAB'.A35>       Auto Repair Automotive          
      53      100 <Cell 'IAB'.A36>       Auto Safety Automotive          
      57      100 <Cell 'IAB'.A37>        Auto Shows Automotive          
      57       75 <Cell 'IAB'.A174>        Graduation Personal Celebrations & Life Events
      53      100 <Cell 'IAB'.A486>       Auto Racing Sports              
      60       75 <Cell 'IAB'.A521>            Beauty Style & Fashion     
      55       75 <Cell 'IAB'.A601>           Laptops Technology & Computing


In [141]:
Ratios('food', 40)

   Ratio Partial_R        location       description group               
      47      100 <Cell 'IAB'.A92>     Food Industry Business and Finance
      50      100 <Cell 'IAB'.A200>      Food & Drink Food & Drink        
      44      100 <Cell 'IAB'.A209>    Food Allergies Food & Drink        
      44      100 <Cell 'IAB'.A210>    Food Movements Food & Drink        
      44       50 <Cell 'IAB'.A493>             Rodeo Sports              


In [145]:
Ratios("hardware", 70)

   Ratio Partial_R        location       description group               


**process.extract** actually uses WRatio() by default, which is a weighted combination of the four fuzz ratios.
you can manually specify the string comparison function via the scorer argument to extract

- base_ratio: The Levenshtein Distance of two string.
- partial_ratio: The ratio of most similar substring.
- token_sort_ratio: Measure of the sequences' similarity sorting the token before comparing.
- token_set_ratio: Find all alphanumeric tokens in each string.

This is an exercise of process.extract for a single word in the **data_IAB_clean2**. The results give the matched name, the score, and the row location of the dataframe.

In [147]:
test = process.extract('auto', data_IAB_clean2['Text_IAB'], limit = 3)
test

[('automotive', 90, 0),
 ('auto body styles automotive', 90, 1),
 ('suv automotive auto body styles', 90, 5)]

In [148]:
test = process.extract('agriculture farm', data_IAB_ps_stem2['Porter'], limit = 3)
test

[('car cultur automot', 65, 24),
 ('agricultur busi financ industri', 62, 96),
 ('pop cultur', 60, 397)]

In [149]:
test = process.extract('auto', data_IAB_lan_stem2['Lancaster'], limit = 3)
test

[('automot', 90, 0),
 ('auto body styl automot', 90, 1),
 ('sed automot auto body styl', 90, 3)]

it works, but I get a match of each row of df with each element of df2
however, I want only the best 3 options

In [ ]:
'''# example found, and modified, but it did not produce what I was expected
# I got a df with a match for each target word


import pandas as pd
df = pd.DataFrame({"Name" : ["Google","google.inc"]})
df2 = pd.DataFrame({"Name" : ["google","google"]})

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

get_match = []
for row in df.index:
    name1 = []
    name1.append(df.get_value(row,"Name"))
    for columns in df2.index:
        name2 = []
        name2.append(df2.get_value(columns,"Name") )
        matched_token=[process.extract(x, name2, limit=3)[0][1] for x in name1]
        get_match.append([matched_token, name1[0], name2[0]])
df_maneet = pd.DataFrame({'name1': [i[1] for i in get_match], 'name2':[i[2] for i in get_match], 'Ratio': [i[0][0] for i in get_match]})'''


In [ ]:
df = data_cF_clean2['Column F']
df2 = data_IAB_clean2['Text_IAB']

In [ ]:
get_match = []
for row in df.index:
    name1 = []
    name1.append(df._get_value(row,"Column F"))
    for columns in df2.index:
        name2 = []
        name2.append(df2._get_value(columns,"Name") )
        matched_token=[process.extract(x, name2, limit=3)[0][1] for x in name1]
        get_match.append([matched_token, name1[0], name2[0]])
df_output_all = pd.DataFrame({'name1': [i[1] for i in get_match], 'name2':[i[2] for i in get_match], 'Ratio': [i[0][0] for i in get_match]})


In [ ]:
df_output_all

In [ ]:
df_output_all.to_excel('First_match.xlsx', sheet_name = 'Test', index= False, freeze_panes=(1,1))

In [ ]:
def get_all_match_score(df1, df2):
    '''
    this function is not working with dataframes NO concatenated.
    The function provides an score match of each word in df1 with df2.
    
    df1 is the dataframe were are the words to be matched, ie data_cF_clean2['Column F']
    df2 is the dataframe that contains the strings to compare for the matching ie data_IAB_clean2['Text_IAB']
    df_output_name is the output dataframe of this function'''
    
    get_match = []
    for row in df1.index:
        name1 = []
        name1.append(df1._get_value(row,"Column F"))
        for columns in df2.index:
            name2 = []
            name2.append(df2._get_value(columns,"Name") )
            matched_token=[process.extract(x, name2, limit=3)[0][1] for x in name1]
            get_match.append([matched_token, name1[0], name2[0]])
    df_output = pd.DataFrame({'Word to be matched': [i[1] for i in get_match], 'Target words':[i[2] for i in get_match], 'Score': [i[0][0] for i in get_match]})
    return df_output
        

In [ ]:
get_all_match_score(data_cF_ps_stem2['Porter'], data_IAB_ps_stem2['Porter'])


In [ ]:
show_dataframes('IAB')

In [ ]:
df = data_cF_lan_stem2
df2 = data_IAB_lan_stem2

In [ ]:
df.head() , df2.head()

This is how I created the matching that gives the 3 best options, using the following two examples code and modified

In [ ]:
'''
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
  
# creating the dictionaries
dict1 = {'name': ["mango", "coco", "choco", "peanut", "apple"]}
dict2 = {'name': ["mango fruit", "coconut", "chocolate",
                  "mangoes", "chocos", "peanuts", "appl"]}
  
# converting to pandas dataframes
dframe1 = pd.DataFrame(dict1)
dframe2 = pd.DataFrame(dict2)
  
# empty lists for storing the matches later
mat1 = []
mat2 = []
p = []
  
# printing the pandas dataframes
print("First dataframe:\n", dframe1, 
      "\nSecond dataframe:\n", dframe2)
  
# converting dataframe column to list
# of elements
# to do fuzzy matching
list1 = dframe1['name'].tolist()
list2 = dframe2['name'].tolist()
  
# taking the threshold as 82
threshold = 82
  
# iterating through list1 to extract 
# it's closest match from list2
for i in list1:
    mat1.append(process.extract(i, list2, limit=2))
dframe1['matches'] = mat1
  
# iterating through the closest matches
# to filter out the maximum closest match
for j in dframe1['matches']:
    for k in j:
        if k[1] >= threshold:
            p.append(k[0])
    mat2.append(",".join(p))
    p = []
  
  
# storing the resultant matches back to dframe1
dframe1['matches'] = mat2
print("\nDataFrame after Fuzzy matching:")
dframe1
'''

In [ ]:
def get_3best_match(dframe1, dframe2, threshold = 80):
    # dframe1 = data_cF_clean2['Column F']
    # dframe2 = data_IAB_clean2['Text_IAB']
    
    # empty lists for storing the matches later
    mat1 = []
    mat2 = []
    # print dataframes.head()
    print('DataFrame1 is:\n', dframe1.head(), '\nDataFrame2:\n', dframe2.head())
    
    # convert df to list of elements to do fuzzy matching
    list1 = dframe1.values.tolist()
    list2 = dframe2.values.tolist()
    
    # iterating through list 1 to extract the closest matches from list 2
    for i in list1:
        mat1.append(process.extract(i, list2, limit=3))
    first_match = []
    for i in range(0,len(dframe1)):
        temp = pd.DataFrame({'first match'})
        if mat1[i][0][1] > threshold:
            first_match.append(mat1[i][0])
        else:
            first_match.append(' ')
    second_match = []
    for i in range(0,len(dframe1)):
        temp = pd.DataFrame({'second match'})
        if mat1[i][1][1] > threshold:
            second_match.append(mat1[i][1])
        else:
            second_match.append(' ')
    third_match = []
    for i in range(0,len(dframe1)):
        temp = pd.DataFrame({'third match'})
        if mat1[i][2][1] > threshold:
            third_match.append(mat1[i][2])
        else:
            third_match.append(' ')
    matches = pd.DataFrame({'Word to match': dframe1, 'First match': first_match, 'Second match': second_match, 'Third match': third_match})
    return matches
    


DataFrame usando preprocessed colF and preprocessed IAB **cF_IAB**

In [ ]:
cF_IAB = get_3best_match(data_cF_clean2['Column F'],data_IAB_clean2['Text_IAB'])
cF_IAB.head(40)

DataFrame matching preprocessed Porter stem colF and preprocessed IAB **cFps_IAB**

In [ ]:
cFps_IAB = get_3best_match(data_cF_ps_stem2['Porter'],data_IAB_clean2['Text_IAB'])
cFps_IAB.head(20)

DataFrame matching preprocessed Lancaster stem colF and preprocessed IAB **cFlan_IAB**

In [ ]:
cFlan_IAB = get_3best_match(data_cF_lan_stem2['Lancaster'],data_IAB_clean2['Text_IAB'])
cFlan_IAB.head(20)

DataFrame matching preprocessed Porter stemmed colF  and preprocessed Poster stemmed IAB **cFps_IABps**

In [ ]:
cFps_IABps = get_3best_match(data_cF_ps_stem2['Porter'],data_IAB_ps_stem2['Porter'])
cFps_IABps.head(20)

DataFrame matching preprocessed Lancaster stemmed colF  and preprocessed Lancaster stemmed IAB **cFlan_IABlan**

In [ ]:
cFlan_IABlan = get_3best_match(data_cF_lan_stem2['Lancaster'], data_IAB_lan_stem2['Lancaster'])
cFlan_IABlan.head(20)

DataFrame matching preprocessed full_TASK1 and preprocessed IAB   cF_IAB **TASK1_IAB**

In [ ]:
TASK1_IAB = get_3best_match(data_clean2['Full_text_TASK1'], data_IAB_clean2['Text_IAB'])
TASK1_IAB.head(20)

DataFrame matching preprocessed full_TASK1 Porter stemmed  and preprocessed IAB  **TASK1ps_IAB**


In [ ]:
TASK1ps_IAB = get_3best_match(data_ps_stem2['Porter'], data_IAB_clean2['Text_IAB'])
TASK1ps_IAB.head(20)

DataFrame matching preprocessed full_TASK1 Lancaster stemmed and preprocessed IAB **TASK1lan_IAB**



In [ ]:
TASK1lan_IAB = get_3best_match(data_lan_stem2['Lancaster'] , data_IAB_clean2['Text_IAB'])
TASK1lan_IAB.head(20)

DataFrame matching preprocessed full_TASK1 Porter stemmed and preprocessed Poster stemmed IAB **TASK1ps_IABps**


In [ ]:
TASK1ps_IABps = get_3best_match(data_ps_stem2['Porter'] , data_IAB_ps_stem2['Porter'])
TASK1ps_IABps.head(20)

DataFrame matching preprocessed full_TASK1 Lancaster stemmed   and preprocessed Lancaster stemmed IAB **TASK1lan_IABlan**

In [ ]:
TASK1lan_IABlan = get_3best_match(data_lan_stem2['Lancaster'] , data_IAB_lan_stem2['Lancaster'])
TASK1lan_IABlan.head(20)

Dataframe matching preprocessed cF and cD columns of TASK1 and preprocessed IAB  **cFcD_IAB**

In [ ]:
cFcD_IAB = get_3best_match(data_cF_cD_clean2['ColF ColD'] , data_IAB_clean2['Text_IAB'], threshold = 70)
cFcD_IAB.head(20)

Dataframe matching preprocessed cF and cD columns of TASK1 w/Porter and preprocess IAB  **cFcDps_IAB**

In [ ]:
cFcDps_IAB = get_3best_match(data_cF_cD_ps_stem2['Porter'] , data_IAB_clean2['Text_IAB'], threshold = 70)
cFcDps_IAB.head(20)

 Dataframe matching preprocessed cF and cD columns of TASK1 w/Lancaster and preprocess IAB  **cFcDlan_IAB**

In [ ]:
cFcDlan_IAB = get_3best_match(data_cF_cD_lan_stem2['Lancaster'] , data_IAB_clean2['Text_IAB'], threshold = 70)
cFcDlan_IAB.head(20)

I will save several dataframes into an excel file for further analysis on excel.

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('TASK1_analysis.xlsx', engine='xlsxwriter')

In [ ]:
# Write each dataframe to a different worksheet. 

cF_IAB.to_excel(writer, sheet_name='cF_IAB', index= False, freeze_panes=(1,1))
cFps_IAB.to_excel(writer, sheet_name='cFps_IAB', index= False, freeze_panes=(1,1))
cFlan_IAB.to_excel(writer, sheet_name='cFlan_IAB', index= False, freeze_panes=(1,1))
cFps_IABps.to_excel(writer, sheet_name='cFps_IABps', index= False, freeze_panes=(1,1))
cFlan_IABlan.to_excel(writer, sheet_name='cFlan_IABlan', index= False, freeze_panes=(1,1))
TASK1_IAB.to_excel(writer, sheet_name='TASK1_IAB', index= False, freeze_panes=(1,1))
TASK1ps_IAB.to_excel(writer, sheet_name='TASK1ps_IAB', index= False, freeze_panes=(1,1))
TASK1lan_IAB.to_excel(writer, sheet_name='TASK1lan_IAB', index= False, freeze_panes=(1,1))
TASK1ps_IABps.to_excel(writer, sheet_name='TASK1_IAB', index= False, freeze_panes=(1,1))
TASK1lan_IABlan.to_excel(writer, sheet_name='TASK1_IAB', index= False, freeze_panes=(1,1))
IAB_codes.to_excel(writer, sheet_name = 'IAB_codes', index= False, freeze_panes=(1,1))

writer.save()

In [ ]:
IAB_codes.head()

I will save another excel file for doing an analysis using cols F and D of TASK1 agains IAB (full text)

In [ ]:
# another file with multiple sheets to be created
writer = pd.ExcelWriter('ColF and ColD TASK1_analysis.xlsx', engine='xlsxwriter')
cFcD_IAB.to_excel(writer, sheet_name='cFcD_IAB', index= False, freeze_panes=(1,1))
cFcDps_IAB.to_excel(writer, sheet_name='cFcDps_IAB', index= False, freeze_panes=(1,1))
cFcDlan_IAB.to_excel(writer, sheet_name='cFcDlan_IAB', index= False, freeze_panes=(1,1))
writer.save()

### search function

This is simple function to locate a word in the IAB sheet.

In [11]:
def search(word):
    sheet = sh_IAB
    # NACE = sh_NACE
    rows = sheet.max_row  
    print('Finding for: {}'.format(word))
    print('%8s %8s %8s %20s %25s %25s' % 
         ('Start', 'End', 'Code', 'Location', 'Description', 'Group'))
    for i in range(2, rows):
        search = re.search(word.lower(), sheet.cell(row=i,column=1).value.lower())
        if search != None:
            code = sheet.cell(row=i, column=2).value
            location = sheet.cell(row=i, column=1)
            group = sheet.cell(row=i, column=4).value
            description = sheet.cell(row=i, column=1).value
            print('%8d %8d %8s %20s %25s %25s' % 
                 (search.start(), search.end(), code, location, description, group))
            

In [12]:
search('shop')

Finding for: shop
   Start      End     Code             Location               Description                     Group
       8       12      154    <Cell 'IAB'.A155>  Malls & Shopping Centers               Attractions
       4        8      247    <Cell 'IAB'.A237>     Workshops and Classes       Hobbies & Interests
       0        4      473    <Cell 'IAB'.A441>                  Shopping                  Shopping
       7       11      475    <Cell 'IAB'.A443>           Flower Shopping                  Shopping
       8       12      477    <Cell 'IAB'.A445>          Grocery Shopping                  Shopping
       8       12      478    <Cell 'IAB'.A446>          Holiday Shopping                  Shopping


### synonym function

Simple function, to find synonyms (I did not manage to embedded into a matching code, several tries)

from nltk.corpus import wordnet

def syn(word):
    #creating a list
    synonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            synonyms.append(lm.name())
    print(set(synonyms))
    
syn('paradise')

Here a code that I tried to use to apply in get match functions. It did not work with the changes I applied.

In [ ]:
'''
# example token and synonym

import nltk
import pandas as pd
import string

sdf = pd.read_excel('C:\synonyms.xlsx')
sd = sdf.apply(lambda x: x.astype(str).str.lower())
words = 'i drove to office everyday in my car'

#######

def tokenize(text):
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    synonym = synonyms(tokens)
    return synonym

def synonyms(words):
    for word in words:
        if(sd[sd['Word'] == word].index.tolist()):
            idx = sd[sd['Word'] == word].index.tolist()
            word = sd.loc[idx]['Synonyms'].item()
        else:
            word
    return word

print(tokenize(words))
'''

below a code I tried to apply on colF of TASK1 to obtain synonyms, I did not work on cells with more than one word. Some issues with the set, and data frames.


In [ ]:
'''

this is not working
cF_syn = pd.DataFrame()   #also I tried cF_syn=[]   
for i in range(0, sh.max_row-1):
    cF_syn = cF_syn.append(syn(data_cF_clean[i]))
    #cF_syn = cF_syn.append({'cF synonyms': syn(data_cF_clean2._get_value(i, 'Column F'))}, ignore_index = True)
    #cF_syn = cF_syn.append({'cF synonyms': syn(data_cF_clean[i])}, ignore_index = True)
    
 '''

In [13]:
from itertools import chain

def get_synonyms(df, column_name, N):
    L = []
    for i in df[column_name]:
        syn = wordnet.synsets(i)
        #flatten all lists by chain, remove duplicates by set
        lemmas = list(set(chain.from_iterable([w.lemma_names() for w in syn])))
        for j in lemmas[:N]:
            #append to final list
            L.append([i, j])
    #create DataFrame
    return (pd.DataFrame(L, columns=['word','syn']))      

In [14]:
get_synonyms(data_cF_clean2, 'Column F', 1).head(10)

NameError: name 'data_cF_clean2' is not defined

### Some thoughts about TASK1 analysis

About the matching:
The fuzzywuzzy library was used, and specifically the proccess.extract function, which consider 4 scenarios (check info above), and I extracted the three best matches. Initially, I did use a threshold of 82, but several rows were empty, and I decided to run the function for several scenarios with threshold of 70.

On the different scenarios, I did metrics to obtain how many rows got a good match (i mean it was important to get the best matching options on each row, but not in all cases the results did make sense). for the several assessed cases.  

My findings show the use of stemming on the IAB is innecesary, it does not give extra value. Therefore, it is not recommended apply stemming on IAB or NACE (for further analysis -TASK2-).

The stemming used on the word to match (TASK1 sheet) was Porter and Lancaster. Lancaster is a more aggresive stemmer and might produce more matches, but because the word is abruptily cut the matches are not asertive. Therefore, a Porter stemming, more conservative approach, is a better option here.

Tested cases:
- ColF of TASK1 and IAB (merge A, D and E columns). The dataFrames were preprocessed ("clean": remove punctuation, digits, stopwords, etc) and no stemming.
- ColF of TASK1 with Porter stemming and IAB (merge A, D and E columns).
- ColF of TASK1 with Lancaster stemming and IAB (merge A, D and E columns).
- ColF of TASK1 with Porter stemming and IAB (merge A, D and E columns) with Porter stemming.
- ColF of TASK1 with Lancaster stemming and IAB (merge A, D and E columns) with Lancaster stemming.
- TASK1 (merge of F, D and H columns) and IAB (merge A, D and E columns).
- TASK1 (merge of F, D and H columns) with Porter stemming and IAB (merge A, D and E columns).
- TASK1 (merge of F, D and H columns) with Lancaster stemming and IAB (merge A, D and E columns).
- etc

The two best matching case correspond to the use of **colF** and **IAB** (IAB is a merge of A, D and E, with not duplicates) with and without Porter stemming. 67 lines of 144 got a correct match, this is equal to **47-48%**. Note: the results are on the same row lines.

The second matching case is using **all columns of TASK1** and **IAB** (IAB is a merge of A, D and E columns) without stemming  (it was not assessed stemming here, since not big changes were seen in previous cases). 65 lines of 144 got a correct match, this is **45%**.

Working together these two cases we got a 94 lines of 144, that represents **65%**.

Around 40% were not picked up for any matching cases, which implies a manual work on them.

In [ ]:
search('seaward')

In [25]:
from nltk.corpus import wordnet

def syn(word):
    # synonym function
    # creating a list
    synonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            synonyms.append(lm.name())
    return synonyms


In [27]:
syn("treaty")

['treaty', 'pact', 'accord']